In [1]:
from MPCRecord import MPCRecord
from BeautifulSoup import BeautifulSoup as bs
import ephem
import os
import requests
from datetime import date, time, datetime

# Render our plots inline
%matplotlib inline

import pandas as pd
import matplotlib.pyplot as plt

pd.set_option('display.mpl_style', 'default') # Make the graphs a bit prettier
plt.rcParams['figure.figsize'] = (15, 5)

In [3]:
obss = pd.read_csv('wsdiff_S82_lon_gt_minus2_obs.csv', parse_dates=['date_obs','date_added'])
links = pd.read_csv('wsdiff_S82_lon_gt_minus2_link.csv')
orbits = pd.read_csv('wsdiff_S82_lon_gt_minus2_orbit.csv')

lobss = pd.merge(links,obss, how='inner',on='objid')
tmpids = pd.Series(lobss.index.values)
tmpids = tmpids.apply(lambda x: str(x))

lobss['tmpid'] = tmpids.str.pad(6, fillchar = '0')
lobss = lobss.groupby('id')

NameError: name 'pd' is not defined

In [789]:
responseframe = pd.DataFrame(columns = ['tmpid','designation','ra-hms','dec-dms','v','offset_ra','offset_dec', 'motion_ra', \
                                    'motion_dec', 'orbit', 'comment'])

In [805]:
i = 0
responses = responseframe
for name, group in lobss:
    observations = ''
    tempgp = lobss.get_group(name)
    for ind, row in tempgp.iterrows():
        q = MPCRecord(MPprovisional=row['tmpid'], obsdate=ephem.date(row['date_obs']), \
                          ra_obs_J2000=row['ra'], dec_obs_J2000=row['dec'])
        observations = observations + q.record + '%0A'

    observations = observations[:-3]

    resoup = retrieveBatch(observations)
    print "got batch " + str(i)
    print "going to parse now"
    #responses = responses.append(parseResponses(resoup, responseframe), ignore_index=True)
    parsed = parseResponses(resoup, responseframe)
    print "back from parsing"
    print "parsed batch " + str(i)
    print parsed
    print "responses is " + str(len(responses)) + " rows long now."
    i = i+1

got batch 0
going to parse now
[u'(380057) 2013 RF81       00 06 58.4 +02 00 32  19.8   3.3W   2.9N    30-    13-    7o  None needed at this time.', u'(104635) 2000 GD117      00 06 58.2 +02 04 12  19.1   3.3W   6.6N    28-    14-   13o  None needed at this time.', u'(164150) 2003 YF119      00 06 50.0 +02 03 45  19.7   5.4W   6.1N    27-     9-   13o  None needed at this time.']
[u'(380057) 2013 RF81       00 06 58.3 +02 00 32  19.8   3.3W   2.9N    30-    13-    7o  None needed at this time.', u'(104635) 2000 GD117      00 06 58.2 +02 04 12  19.1   3.3W   6.5N    28-    14-   13o  None needed at this time.', u'(164150) 2003 YF119      00 06 49.9 +02 03 45  19.7   5.4W   6.1N    27-     9-   13o  None needed at this time.']
[u' (14077) Volfango        00 06 27.8 +01 44 06  18.5   2.3E   7.5S    25-     7-   17o  None needed at this time.', u' (91070) 1998 FO71       00 05 55.7 +01 44 03  17.9   5.7W   7.5S    23-    22-   12o  None needed at this time.']
[u' (14077) Volfango        00

IndexError: list index out of range

In [808]:
enumerate(resps, start = 0)

In [761]:
def retrieveBatch(observations):
    url = "http://www.minorplanetcenter.net/cgi-bin/mpcheck.cgi"

    today=datetime.utcnow()
    year=str(today.year)
    month=str(today.strftime("%m"))
    day=str(round(today.day + (float(today.hour)/24 + float(today.minute)/(24*60)),2))

    limit='20.0'
    radius='10'

    payload = "TextArea=" + observations + "&day=" + day + "&decl=&limit=" + limit + "&month=" + month +  \
              "&mot=h&needed=f&oc=W84&pdes=u&ps=n&ra=&radius=" + radius + "&sort=d&tmot=s&type=p&which=obs&year=" + year
    headers = {
        'accept': "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8",
        'origin': "http//www.minorplanetcenter.net",
        'upgrade-insecure-requests': "1",
        'user-agent': "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_3) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/48.0.2564.116 Safari/537.36",
        'content-type': "application/x-www-form-urlencoded",
        'referer': "http//www.minorplanetcenter.net/cgi-bin/checkmp.cgi",
        'accept-encoding': "gzip, deflate",
        'accept-language': "en-US,en;q=0.8",
        'cookie': "WT_FPC=id=68.56.146.120-2763983344.30502161lv=1456166666520ss=1456166666520; __utmt=1; __utma=108877598.394446093.1456105075.1456105075.1456166667.2; __utmb=108877598.1.10.1456166667; __utmc=108877598; __utmz=108877598.1456105075.1.1.utmcsr=google|utmccn=(organic)|utmcmd=organic|utmctr=(not%20provided)"
        }

    response = requests.request("POST", url, data=payload, headers=headers)

    resoup = bs(response.text)
    return resoup

In [767]:
def parseResponses(resoup, frame):
    responseframe = frame
    resps = resoup.findAll('pre')
    for index, item in enumerate(resps, start = 0):
        if resps[index].text[0:6] == "Object":
            continue
        if resps[index+1].text[0:6] == "Object":
            tmpid = resps[index].text[0:6]
            matches = resps[index+1].text[204:]
            matches = matches.splitlines()
            print matches
            matchframe = buildmatch(matches)
            responseframe = responseframe.append(matchframe,ignore_index = True)
            continue
        #so if it all falls out, write a disappointed row    
        else:
            tmpid = resps[index].text[0:6]
            matches = ["-" * 125]
            matchframe = buildmatch(matches)
            responseframe = responseframe.append(matchframe,ignore_index = True)

    return responseframe

In [804]:
resps

[<pre>
      000466   C2013 09 30.14987300 07 11.53  01 57 39.0               r      W84
 </pre>, <pre>
  Object designation         R.A.      Decl.     V       Offsets     Motion/hr   Orbit  <a href="http://www.cfa.harvard.edu/iau/info/FurtherObs.html">Further observations?</a>
                            h  m  s     &#176;  '  "        R.A.   Decl.  R.A.  Decl.        Comment (Elong/Decl/V at date 1)
 
 (380057) 2013 RF81       00 06 58.4 +02 00 32  19.8   3.3W   2.9N    30-    13-    7o  None needed at this time.
 (104635) 2000 GD117      00 06 58.2 +02 04 12  19.1   3.3W   6.6N    28-    14-   13o  None needed at this time.
 (164150) 2003 YF119      00 06 50.0 +02 03 45  19.7   5.4W   6.1N    27-     9-   13o  None needed at this time.
  </pre>, <pre>
      000467   C2013 09 30.15134300 07 11.52  01 57 38.8               r      W84
 </pre>, <pre>
  Object designation         R.A.      Decl.     V       Offsets     Motion/hr   Orbit  <a href="http://www.cfa.harvard.edu/iau/info/Furt

In [742]:
def buildmatch(matchtext):
    matchframe = pd.DataFrame({'matchtext': matches})

    matchframe['tmpid'] = tmpid
    matchframe['designation'] = matchframe.matchtext.str[0:26]
    matchframe['ra-hms'] = matchframe.matchtext.str[26:36]
    matchframe['dec-dms'] = matchframe.matchtext.str[36:47]
    matchframe['v'] = matchframe.matchtext.str[47:54]
    matchframe['offset_ra'] = matchframe.matchtext.str[54:61]
    matchframe['offset_dec'] = matchframe.matchtext.str[61:69]
    matchframe['motion_ra'] = matchframe.matchtext.str[69:76]
    matchframe['motion_dec'] = matchframe.matchtext.str[76:81]
    matchframe['orbit'] = matchframe.matchtext.str[81:87]
    matchframe['comment'] = matchframe.matchtext.str[87:]

    del matchframe['matchtext']
    return matchframe

AttributeError: 'DataFrame' object has no attribute 'matchtext'